Rules of the file : 
* If dataset works better than the previous one, keep that one. Else, use the previous one

# Base kit and first functions/columns added - name of the file : data

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

### Loading the data with pandas

First, download the data files from Kaggle, and put them to into the data folder.


Data is stored in [Parquet format](https://parquet.apache.org/), an efficient columnar data format. We can load the train set with pandas,

In [ ]:
data = pd.read_parquet(Path("data") / "train.parquet")
data.head()

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,log_bike_count
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.693147
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.609438
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2.302585


We can check general information about different columns:

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 496827 entries, 48321 to 929187
Data columns (total 12 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   counter_id                 496827 non-null  category      
 1   counter_name               496827 non-null  category      
 2   site_id                    496827 non-null  int64         
 3   site_name                  496827 non-null  category      
 4   bike_count                 496827 non-null  float64       
 5   date                       496827 non-null  datetime64[us]
 6   counter_installation_date  496827 non-null  datetime64[us]
 7   coordinates                496827 non-null  category      
 8   counter_technical_id       496827 non-null  category      
 9   latitude                   496827 non-null  float64       
 10  longitude                  496827 non-null  float64       
 11  log_bike_count             496827 non-null  float64  

and in particular the number of unique entries in each column,

In [ ]:
data.nunique(axis=0)

counter_id                     56
counter_name                   56
site_id                        30
site_name                      30
bike_count                    998
date                         8974
counter_installation_date      22
coordinates                    30
counter_technical_id           30
latitude                       30
longitude                      30
log_bike_count                998
dtype: int64

We have a 30 counting sites where sometimes multiple counters are installed per location.  Let's look at the most frequented stations:

In [ ]:
(
    data.groupby(["site_name", "counter_name"])["bike_count"].sum()
    .sort_values(ascending=False)
    .head(10)
    .to_frame()
)

/var/folders/dr/m9v4shms079d9s6z6c8cg57m0000gn/T/ipykernel_62292/1192055722.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data.groupby(["site_name", "counter_name"])["bike_count"].sum()


,,bike_count
site_name,counter_name,
Totem 73 boulevard de Sébastopol,Totem 73 boulevard de Sébastopol S-N,1992227.0
Totem 64 Rue de Rivoli,Totem 64 Rue de Rivoli O-E,1551730.0
Totem 73 boulevard de Sébastopol,Totem 73 boulevard de Sébastopol N-S,1497912.0
67 boulevard Voltaire SE-NO,67 boulevard Voltaire SE-NO,1130565.0
Totem 64 Rue de Rivoli,Totem 64 Rue de Rivoli E-O,1015875.0
27 quai de la Tournelle,27 quai de la Tournelle SE-NO,984914.0
Quai d'Orsay,Quai d'Orsay E-O,948357.0
Totem Cours la Reine,Totem Cours la Reine O-E,894937.0
Face au 48 quai de la marne,Face au 48 quai de la marne SO-NE,884907.0


### Feature extraction

To account for the temporal aspects of the data, we cannot input the `date` field directly into the model. Instead we extract the features on different time-scales from the `date` field:

In [ ]:
def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

In [ ]:
data["date"].head()

48321   2020-09-01 02:00:00
48324   2020-09-01 03:00:00
48327   2020-09-01 04:00:00
48330   2020-09-01 15:00:00
48333   2020-09-01 18:00:00
Name: date, dtype: datetime64[us]

In [ ]:
_encode_dates(data[["date"]].head())

,year,month,day,weekday,hour
48321,2020,9,1,1,2
48324,2020,9,1,1,3
48327,2020,9,1,1,4
48330,2020,9,1,1,15
48333,2020,9,1,1,18


To use this function with scikit-learn estimators we wrap it with [FunctionTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html):

In [ ]:
from sklearn.preprocessing import FunctionTransformer

date_encoder = FunctionTransformer(_encode_dates, validate=False)
sample_encoded = date_encoder.fit_transform(data[["date"]]).head()
sample_encoded

,year,month,day,weekday,hour
48321,2020,9,1,1,2
48324,2020,9,1,1,3
48327,2020,9,1,1,4
48330,2020,9,1,1,15
48333,2020,9,1,1,18


Since it is unlikely that, for instance, that `hour` is linearly correlated with the target variable, we would need to additionally encode categorical features for linear models. This is classically done with [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html), though other encoding strategies exist.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse_output=False)

enc.fit_transform(sample_encoded[["hour"]])

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [ ]:
pip install holidays

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import holidays
fr_holidays = holidays.France(years=[2020,2021])

def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the Date columns
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour
    X['bankholidays'] = X['date'].isin(fr_holidays).astype(int)
    X['weekend'] = X['weekday'].isin([6,7]).astype(int)
    X['night'] = X['hour'].isin([20, 21, 22, 23, 24, 1, 2, 3, 4, 5, 6]).astype(int)
    X['morning'] = X['hour'].isin([6, 7, 8, 9, 10, 11, 12]).astype(int)
    X['midday'] = X['hour'].isin([12, 13, 14]).astype(int)
    X['evening'] = X['hour'].isin([14, 15, 16, 17, 18, 19, 20]).astype(int)

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

In [ ]:
_encode_dates(data_bike)

/var/folders/dr/m9v4shms079d9s6z6c8cg57m0000gn/T/ipykernel_69161/2368337008.py:12: FutureWarning: The behavior of 'isin' with dtype=datetime64[us] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  X['bankholidays'] = X['date'].isin(fr_holidays).astype(int)


,site_id,latitude,longitude,counter_id,counter_name,site_name,coordinates,counter_technical_id,counter_installation_date,year,month,day,weekday,hour,bankholidays,weekend,night,morning,midday,evening
48321,100007049,48.846028,2.375429,100007049-102007049,28 boulevard Diderot E-O,28 boulevard Diderot,"48.846028,2.375429",Y2H15027244,2013-01-18,2020,9,1,1,2,0,0,1,0,0,0
48324,100007049,48.846028,2.375429,100007049-102007049,28 boulevard Diderot E-O,28 boulevard Diderot,"48.846028,2.375429",Y2H15027244,2013-01-18,2020,9,1,1,3,0,0,1,0,0,0
48327,100007049,48.846028,2.375429,100007049-102007049,28 boulevard Diderot E-O,28 boulevard Diderot,"48.846028,2.375429",Y2H15027244,2013-01-18,2020,9,1,1,4,0,0,1,0,0,0
48330,100007049,48.846028,2.375429,100007049-102007049,28 boulevard Diderot E-O,28 boulevard Diderot,"48.846028,2.375429",Y2H15027244,2013-01-18,2020,9,1,1,15,0,0,0,0,0,1
48333,100007049,48.846028,2.375429,100007049-102007049,28 boulevard Diderot E-O,28 boulevard Diderot,"48.846028,2.375429",Y2H15027244,2013-01-18,2020,9,1,1,18,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929175,300014702,48.839770,2.301980,300014702-353245971,254 rue de Vaugirard SO-NE,254 rue de Vaugirard,"48.83977,2.30198",Y2H20114504,2020-11-29,2021,9,9,3,6,0,0,1,1,0,0
929178,300014702,48.839770,2.301980,300014702-353245971,254 rue de Vaugirard SO-NE,254 rue de Vaugirard,"48.83977,2.30198",Y2H20114504,2020-11-29,2021,9,9,3,10,0,0,0,1,0,0
929181,300014702,48.839770,2.301980,300014702-353245971,254 rue de Vaugirard SO-NE,254 rue de Vaugirard,"48.83977,2.30198",Y2H20114504,2020-11-29,2021,9,9,3,15,0,0,0,0,0,1
929184,300014702,48.839770,2.301980,300014702-353245971,254 rue de Vaugirard SO-NE,254 rue de Vaugirard,"48.83977,2.30198",Y2H20114504,2020-11-29,2021,9,9,3,22,0,0,1,0,0,0


# Getting rid of some data - name of the file : data_bike_rid

In [ ]:
target_columns = ["bike_count", "log_bike_count"]
numerical_columns = ['latitude', 'longitude']
categorical_columns = ['counter_name', 'site_name', 'counter_technical_id']
date_columns = ['date', 'counter_installation_date']
all_columns = numerical_columns + categorical_columns + date_columns + target_columns

data_bike_rid = data[all_columns]
# cosmetic changes

# Adding the arrondissement - name of the file : data_bike_arr

In [ ]:
data_bike_arr = data_bike_rid.copy()

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Load GeoJSON file for Paris arrondissements
gdf = gpd.read_file(Path("external_data") / "arrondissements.geojson")

# Convert coordinates to Points
data_bike_arr['geometry'] = data.apply(lambda row: Point(row['latitude'], row['longitude']), axis=1)
geo_df = gpd.GeoDataFrame(data, geometry='geometry')
geo_df = geo_df.set_crs(epsg=4326, allow_override=True)  # Set the CRS


data_bike_arr.head()

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,log_bike_count,geometry,Arrondissement
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000,POINT (48.846028 2.375429),NaN
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.693147,POINT (48.846028 2.375429),NaN
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000,POINT (48.846028 2.375429),NaN
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.609438,POINT (48.846028 2.375429),NaN
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2.302585,POINT (48.846028 2.375429),NaN


In [ ]:
geo_df.head()

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,log_bike_count,geometry,Arrondissement
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000,POINT (48.84603 2.37543),NaN
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.693147,POINT (48.84603 2.37543),NaN
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000,POINT (48.84603 2.37543),NaN
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.609438,POINT (48.84603 2.37543),NaN
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2.302585,POINT (48.84603 2.37543),NaN


In [ ]:
gdf.head()

,n_sq_ar,c_ar,c_arinsee,l_ar,l_aroff,n_sq_co,surface,perimetre,geom_x_y,geometry
0,750000001,1,75101,1er Ardt,Louvre,750001537,1.824613e+06,6054.936862,"{ ""lon"": 2.3364433620533878, ""lat"": 48.8625627...","POLYGON ((2.32801 48.86992, 2.32997 48.86851, ..."
1,750000003,3,75103,3ème Ardt,Temple,750001537,1.170883e+06,4519.263648,"{ ""lon"": 2.3600009858976918, ""lat"": 48.8628723...","POLYGON ((2.36383 48.8675, 2.36389 48.86747, 2..."
2,750000014,14,75114,14ème Ardt,Observatoire,750001537,5.614877e+06,10317.483310,"{ ""lon"": 2.3265420441989466, ""lat"": 48.8292445...","POLYGON ((2.33381 48.84061, 2.33673 48.83965, ..."
3,750000004,4,75104,4ème Ardt,Hôtel-de-Ville,750001537,1.600586e+06,5420.908434,"{ ""lon"": 2.3576296203249969, ""lat"": 48.8543414...","POLYGON ((2.36851 48.85573, 2.369 48.85374, 2...."
4,750000012,12,75112,12ème Ardt,Reuilly,750001537,1.631478e+07,24089.666298,"{ ""lon"": 2.4213235156144646, ""lat"": 48.8349741...","POLYGON ((2.41388 48.83357, 2.41401 48.83357, ..."


In [ ]:
merged = gpd.sjoin(geo_df, gdf, how="left", predicate="within")
merged.head()

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,...,index_right,n_sq_ar,c_ar,c_arinsee,l_ar,l_aroff,n_sq_co,surface,perimetre,geom_x_y
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Perform spatial join to find arrondissement
result = gpd.sjoin(geo_df, gdf, how='left', predicate='within')

# Add arrondissement name to the DataFrame
data_bike_arr['Arrondissement'] = result['c_ar']

data.head()

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,log_bike_count,geometry,Arrondissement
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000,POINT (48.846028 2.375429),NaN
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.693147,POINT (48.846028 2.375429),NaN
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000,POINT (48.846028 2.375429),NaN
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.609438,POINT (48.846028 2.375429),NaN
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2.302585,POINT (48.846028 2.375429),NaN


# Adding the external meteorological data : data_bike_meteo

In [ ]:
external_data = pd.read_csv((Path("data") / "external_data.csv"))

In [ ]:
external_data.head()

,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,u,...,hnuage1,nnuage2,ctype2,hnuage2,nnuage3,ctype3,hnuage3,nnuage4,ctype4,hnuage4
0,7149,2021-01-01 00:00:00,100810,80,1,270,1.8,272.75,272.15,96,...,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7149,2021-01-01 03:00:00,100920,110,3,300,1.7,271.25,270.95,98,...,1500.0,2.0,3.0,3000.0,NaN,NaN,NaN,NaN,NaN,NaN
2,7149,2021-01-01 06:00:00,100950,30,3,290,2.6,271.95,271.65,98,...,480.0,4.0,6.0,2000.0,6.0,3.0,3000.0,NaN,NaN,NaN
3,7149,2021-01-01 09:00:00,101100,150,2,280,1.7,272.45,272.05,97,...,1740.0,3.0,3.0,2800.0,NaN,NaN,NaN,NaN,NaN,NaN
4,7149,2021-01-01 12:00:00,101110,30,0,50,1.0,276.95,274.15,82,...,330.0,4.0,6.0,570.0,7.0,6.0,810.0,NaN,NaN,NaN
